In [13]:
from arcgis.gis import GIS
from arcgis.geocoding import geocode
from arcgis.raster.functions import *
from arcgis import geometry
from IPython.display import display
    
import pandas as pd

# connect as an anonymous user
gis = GIS()

# search for the landsat multispectral imagery layer
landsat_item = gis.content.search('"Landsat Multispectral"', 'Imagery Layer', outside_org=True)[0]
landsat = landsat_item.layers[0]
df = None

In [14]:
import ipywidgets as widgets

# text box widget
location = widgets.Text(value='Ranchi, India', placeholder='Ranchi, India',
                        description='Location:', disabled=False)

# command button widget
gobtn = widgets.Button(description='Go', disabled=False,
                       button_style='', tooltip='Go', icon='check')

# define what happens whent the command button is clicked
def on_gobutton_clicked(b):
    global df
    global m
    global oldslider
    
    # geocode the place name and set that as the map's extent
    area = geocode(location.value)[0]
    m.extent = area['extent']
    df = filter_images()
    
gobtn.on_click(on_gobutton_clicked)

location_items = [location, gobtn]
widgets.HBox(location_items)

In [15]:
m = gis.map(location.value)
m.add_layer(landsat)
display(m)

In [16]:
oldindex =  0 # int(len(df)/2)

# before image date slider
oldslider = widgets.IntSlider(value=oldindex, min=0,max=10, #len(df) - 1,
                              step=1, description='Older:', disabled=False,
                              continuous_update=True, orientation='horizontal',
                              readout=False, readout_format='i', slider_color='white')

old_label = widgets.Label(value='')#str(df.Time.iloc[oldindex].date()))

# define the slider behavior
def on_old_value_change(change):
    global df
    i = change['new']
    if df is not None:
        try:
            # print(df.Time.iloc[i].date())
            old_label.value = str(df.Time.iloc[i].date())
        except:
            pass
        
oldslider.observe(on_old_value_change, names='value')    
widgets.HBox([oldslider, old_label])

In [17]:
newindex = 0 # len(df) - 1

# after image date slider
newslider = widgets.IntSlider(value=newindex, min=0, max=10, #len(df) - 1,
                              step=1, description='Newer:', disabled=False,
                              continuous_update=True, orientation='horizontal',
                              readout=False, readout_format='i', slider_color='white')

new_label = widgets.Label(value='') #str(df.Time.iloc[newindex].date()))

# define the slider behavior
def on_new_value_change(change):
    global df
    i = change['new']
    if df is not None:
        try:
        # print(df.Time.iloc[i].date())
            new_label.value = str(df.Time.iloc[i].date())
        except:
            pass
newslider.observe(on_new_value_change, names='value')
widgets.HBox([newslider, new_label])

In [18]:
def update_sliders(tdf):
    global oldslider
    global newslider
    
    oldslider.max = len(tdf) - 1
    newslider.max = len(tdf) -1
    oldindex = int(len(tdf)/2)
    newindex = int(len(tdf) -1)
    oldslider.value = oldindex
    newslider.value = newindex
    old_label.value = str(tdf.Time.iloc[oldindex].date())
    new_label.value = str(tdf.Time.iloc[newindex].date())


def filter_images():
    global df
    area = geocode(location.value, out_sr=landsat.properties.spatialReference)[0]
    extent = area['extent']

    selected = landsat.filter_by(where="(Category = 1) AND (CloudCover <=0.10)", 
                             geometry=geometry.filters.intersects(extent))
    fs = selected.query(out_fields="AcquisitionDate, GroupName, Best, CloudCover, WRS_Row, WRS_Path, Month, Name", 
                  return_geometry=True,
                  return_distinct_values=False,
                  order_by_fields="AcquisitionDate")
    tdf = fs.df
    df = tdf
    tdf['Time'] = pd.to_datetime(tdf['AcquisitionDate'], unit='ms')    
    
    if len(tdf) > 1:
        update_sliders(tdf)

    # m.draw(tdf.iloc[oldslider.value].SHAPE)
    
    return tdf

In [19]:
df = filter_images(); df

,AcquisitionDate,Best,CloudCover,GroupName,Month,Name,OBJECTID,Shape_Area,Shape_Length,WRS_Path,WRS_Row,SHAPE,Time
0,187228800000,95849044,-0.0100,p151r044_2x19751208,12,p151r044_2dm19751208_z45_MS,593431,4.034064e+10,804604.824167,151,44,"{'rings': [[[9571542.173799999, 2737083.279300...",1975-12-08 00:00:00.000
1,720576000000,91860044,-0.0100,p140r044_5x19921101,11,p140r044_5dt19921101_z45_MS,585190,3.858019e+10,793610.663058,140,44,"{'rings': [[[9656510.8985, 2731066.4963000007]...",1992-11-01 00:00:00.000
2,1004659200000,88860044,0.0000,p140r044_7x20011102,11,p140r044_7dt20011102_z45_MS,576928,3.699148e+10,770053.331093,140,44,"{'rings': [[[9660501.5825, 2724625.0896999985]...",2001-11-02 00:00:00.000
3,1131148800000,79860044,0.0000,L5140044_04420051105,11,L5140044_04420051105_MS,562789,3.816735e+10,782430.592832,140,44,"{'rings': [[[9668660.803, 2720957.0716999993],...",2005-11-05 00:00:00.000
4,1256169600000,72859044,0.0000,L5141044_04420091022,10,L5141044_04420091022_MS,554613,3.831913e+10,784012.708911,141,44,"{'rings': [[[9493653.1199, 2721864.136599999],...",2009-10-22 00:00:00.000
5,1256256000000,72860044,0.0000,L7140044_04420091023,10,L72140044_04420091023_MS,558574,3.838273e+10,784570.785117,140,44,"{'rings': [[[9666225.191, 2723690.9299000017],...",2009-10-23 00:00:00.000
6,1384145105786,42860044,0.0001,LC81400442013315LGN00_MTL,11,LC81400442013315LGN00,45502,4.098184e+10,810122.465955,140,44,"{'rings': [[[9663540.511300001, 2726647.231800...",2013-11-11 04:45:05.786
7,1411793008246,28860044,0.0005,LC81400442014270LGN00_MTL,9,LC81400442014270LGN00,29137,4.109451e+10,810445.186431,140,44,"{'rings': [[[9667349.028700002, 2726525.537000...",2014-09-27 04:43:28.246
8,1414557811599,29860044,0.0153,LC81400442014302LGN00_MTL,10,LC81400442014302LGN00,20961,4.099270e+10,810225.858533,140,44,"{'rings': [[[9666363.5324, 2726650.4217000008]...",2014-10-29 04:43:31.599
9,1415940215130,32860044,0.0397,LC81400442014318LGN00_MTL,11,LC81400442014318LGN00,19029,4.100708e+10,810369.079234,140,44,"{'rings': [[[9664271.855700001, 2726647.769400...",2014-11-14 04:43:35.130


In [20]:
# create the action button
diffbtn = widgets.Button(description='Detect changes', disabled=False,
                         button_style='success', tooltip='Show Different Image',
                         icon='check')

def on_diffbutton_clicked(b):
    # m.clear_graphics()
    first = df.iloc[oldslider.value].OBJECTID
    last = df.iloc[newslider.value].OBJECTID
    old = landsat.filter_by('OBJECTID='+str(first))
    new = landsat.filter_by('OBJECTID='+str(last))
    diff = stretch(composite_band([ndvi(old, '5 4'),
                               ndvi(new, '5 4'),
                               ndvi(old, '5 4')]), 
                               stretch_type='stddev',  num_stddev=3, min=0, max=255, dra=True, astype='u8')
    m.add_layer(diff)
    
diffbtn.on_click(on_diffbutton_clicked)
diffbtn